# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [1]:
! pip install evaluate

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from datasets import load_dataset

from peft import LoraConfig, TaskType
from peft import get_peft_model
from peft import AutoPeftModelForSequenceClassification 

from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import torch
import torch.nn.functional as F

import numpy as np
import pandas as pd
import evaluate

from workspace_utils import active_session

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [4]:
# Dataset
ds = load_dataset("medalpaca/medical_meadow_health_advice", split='train')
ds = ds.remove_columns("instruction")

mapping = {"This is no advice":0, "This is a weak advice":1, "This is a strong advice":2}
ds = ds.add_column("labels", [mapping[i] for i in ds["output"]])

ds = ds.train_test_split(test_size=0.25)
ds

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'labels'],
        num_rows: 6507
    })
    test: Dataset({
        features: ['output', 'input', 'labels'],
        num_rows: 2169
    })
})

In [5]:
labels = [0,1,2]
label_count = len(labels)

id2label = {0:"No advice", 1:"Weak advice", 2:"Strong advice"}
label2id = {"No advice":0, "Weak advice":1, "Strong advice":2}

label_count

3

In [6]:
# Tokenization
tokenizer = AutoTokenizer.from_pretrained("gpt2")

if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token
    
    tokenized_dataset = {}
tokenized_dataset["train"] = ds["train"].map(lambda x: tokenizer(x["input"], padding="max_length", truncation=True), batched=True)
tokenized_dataset["test"] = ds["test"].map(lambda x: tokenizer(x["input"], padding="max_length", truncation=True), batched=True)

Map:   0%|          | 0/6507 [00:00<?, ? examples/s]

Map:   0%|          | 0/2169 [00:00<?, ? examples/s]

In [7]:
# Foundation Model without changing out_features in the last Linear layer
foundation = AutoModelForSequenceClassification.from_pretrained("gpt2")
foundation.config.pad_token_id = tokenizer.pad_token_id

# Foundation model with changing out_features to 3 in the last Linear layer
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=label_count, id2label=id2label, label2id=label2id)
model.config.pad_token_id = tokenizer.pad_token_id

# to cuda
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
foundation.to(device)
model.to(device)

model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)

In [8]:
test_df = pd.DataFrame(ds["test"])

# formulate as questions to ask the foundation model and give instuctions
test_df["input_question"] = test_df["input"].apply(lambda x :f"Question: is this a 0:No advice, 1:Weak advice, 2:Strong advice? {x}")
print(test_df["input_question"][1])

Question: is this a 0:No advice, 1:Weak advice, 2:Strong advice? As detailed in Table 2 and Figure 2, this biker position was shown here to reduce myocardial attenuation and corresponding artifacts, as compared with the sitting or supine position, with this reduction mainly involving the basal portion of the inferior area, similarly to that previously reported for the prone versus the supine position. 


In [9]:
def predict(model, text):
    input_tokens =  tokenizer(text,  return_tensors="pt", padding="max_length", truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**input_tokens)
        logits = outputs.logits
        
    probs = torch.nn.functional.softmax(logits, dim=1)
    return probs.argmax().item()

In [10]:
# get predictions from foundation model
test_df["foundation_predictions"] = test_df["input_question"].map(lambda x: predict(foundation,x))

In [11]:
accuracy_metric = evaluate.load("accuracy")

foundation_results = accuracy_metric.compute(references=test_df["labels"], predictions=test_df["foundation_predictions"])
print(foundation_results)

{'accuracy': 0.7514983863531581}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
# convert to PEFT model
config = LoraConfig(task_type=TaskType.SEQ_CLS,inference_mode=False,r=4,lora_alpha=16,lora_dropout=0.1,fan_in_fan_out=True,)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 152,064 || all params: 124,591,872 || trainable%: 0.12204969518396834


In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [10]:
# Fine-tuning using PEFT
training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,

        evaluation_strategy="epoch",
        save_strategy="epoch",
    #     load_best_model_at_end=True,
        num_train_epochs=1,
    #     weight_decay=0.01,    
    #     remove_unused_columns=False 

    )


trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],#train_lora,
        eval_dataset=tokenized_dataset['test'],#test_lora,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )


# with active_session():

trainer.train()

# save fine-tuned PEFT
lora_model.save_pretrained("gpt2-lora")

trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.584300,0.576628,0.756109


{'eval_loss': 0.5766279697418213,
 'eval_accuracy': 0.756108805901337,
 'eval_runtime': 203.3324,
 'eval_samples_per_second': 10.667,
 'eval_steps_per_second': 2.671,
 'epoch': 1.0}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [13]:
ft_lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt2-lora", num_labels=label_count)
ft_lora_model.config.pad_token_id = ft_lora_model.config.eos_token_id
ft_lora_model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_

In [14]:
# inference
x = "not recommended"
predict(ft_lora_model,x)

0

In [15]:
# get predictions from the finetuned model
test_df["finetuned_predictions"] = test_df["input"].map(lambda x: predict(ft_lora_model,x))

In [16]:
# index 2164 has different predictions from foundation model and finetuned model
test_df.tail()

,output,input,labels,input_question,foundation_predictions,finetuned_predictions
2164,This is a strong advice,"Thus, EPA and DHA supplementation should be co...",2,"Question: is this a 0:No advice, 1:Weak advice...",0,0
2165,This is no advice,Meta-analysis showed no significant associatio...,0,"Question: is this a 0:No advice, 1:Weak advice...",0,0
2166,This is a weak advice,MPV could be a promising and easily available ...,1,"Question: is this a 0:No advice, 1:Weak advice...",0,0
2167,This is no advice,Its therapeutic effect is independent of its e...,0,"Question: is this a 0:No advice, 1:Weak advice...",0,0
2168,This is no advice,Nitrous oxide inhalation was preferable to inj...,0,"Question: is this a 0:No advice, 1:Weak advice...",0,0


In [17]:
accuracy_metric = evaluate.load("accuracy")

finetuning_results = accuracy_metric.compute(references=test_df["labels"], predictions=test_df["finetuned_predictions"])
print(finetuning_results)

{'accuracy': 0.7574919317657907}


# There is a little increase in the accuarcy after finetuning.
#  I only did 1 epoch, because on resources limitations and connection problems in the workspace.